In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
n_l = 5
n_h = 3
C = torch.rand(n_l, n_h)
C

In [ ]:
C = torch.Tensor([
    [1, 0.1, 0.1],
    [0.3, 0.3, 0.3],
    [0.1, 0.5, 0.5],
    [0.1, 0.1, 0.9],
    
])

In [ ]:
C_c = torch.softmax(C / (1/n_l), dim=0)
C_c

In [ ]:
C_r = torch.softmax(C / (1/n_h), dim=1)
C_r

In [ ]:
C = C_c * C_r
C

In [ ]:
C.min(), C.max(), C.mean()

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, dim, mult=4, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim, dim * mult * 2),
                                 nn.ReLU(),
                                 nn.Dropout(dropout),
                                 nn.Linear(dim * mult, dim))

    def forward(self, x):
        return self.net(x)

class CrossAttention1(nn.Module):
    def __init__(self, n_z, d_z, d_x, d_i):
        super().__init__()
        self.z = torch.nn.Parameter(torch.rand(n_z, d_z), requires_grad=True)
        self.to_q = nn.Linear(d_z, d_i, bias=False)
        self.to_kv = nn.Linear(d_x, d_i * 2, bias=False)
        self.scale = d_i**-0.5
    
    def forward(self, x):
        q = self.to_q(self.z)
        k, v = self.to_kv(x).chunk(2, dim=-1)
        #
        S = torch.einsum("id, ...jd -> ...ij", q, k) * self.scale
        
        A_l = S.softmax(dim=1)
        A_h = S.softmax(dim=2)
        #
        C = A_l * A_h
        
        # (b,n_h,n_l) (b,n_l,d_i)
        out = torch.einsum("...ij, ...jk -> ...ik", C, v)
        #
        return out

def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = x2.norm(p=2, dim=1, keepdim=True)
    return torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

def pw_cosine_distance(input_a, input_b):
    normalized_input_a = torch.nn.functional.normalize(input_a)  
    normalized_input_b = torch.nn.functional.normalize(input_b)
    res = torch.mm(normalized_input_a, normalized_input_b.T)
    #res *= -1 # 1-res without copy
    #res += 1
    return res
    
class CrossAttention(nn.Module):
    def __init__(self, n_z, d_z, d_x, d_i):
        super().__init__()
        self.z = torch.nn.Parameter(torch.rand(n_z, d_z), requires_grad=True)
        self.to_q = nn.Linear(d_z, d_i, bias=False)
        self.to_kv = nn.Linear(d_x, d_i * 2, bias=False)
        self.scale = d_i**-0.5
    
    def forward(self, x):
        q = self.to_q(self.z)
        k, v = self.to_kv(x).chunk(2, dim=-1)
        #
        S = torch.einsum("id, ...jd -> ...ij", q, k)
        
        C = torch.softmax(S / self.scale, dim=1)
        
        # (b,n_h,n_l) (b,n_l,d_i)
        out = torch.einsum("...ij, ...jk -> ...ik", C, v)
        #
        return q, k, v

In [ ]:
b = 1
n_z = 2
n_x = 3
d_z = 4
d_x = 5
#
d_i = 6
#
x = torch.rand(b, n_x, d_x)
model = CrossAttention(n_z, d_z, d_x, d_i)
q,k,v = model(x)
#
S = torch.einsum("id, ...jd -> ...ij", q, k)
C = torch.softmax(S, dim=1)

In [ ]:
cs = pw_cosine_distance(q, k[0])
cs

In [ ]:
S

In [ ]:
cos = nn.CosineSimilarity(dim=2, eps=1e-6)
cos(q.unsqueeze(0), k)

In [ ]:
k.shape

In [ ]:
a.softmax(dim=0)

In [ ]:
# h,l
S = torch.Tensor([
    [1, 0.1, 0.1],
    [0.3, 0.3, 0.5],
]) * 4
S = S.reshape(1, 2, 3)

In [ ]:
C_l = S.softmax(dim=1)
C_l

In [ ]:
C_h = S.softmax(dim=2)
#
C_h

In [ ]:
        I_l = C_h.sum(axis=1) / C_h.sum(axis=1).sum()
        I_h = C_l.sum(axis=2) / C_l.sum(axis=2).sum()

In [ ]:
I = torch.einsum("...i,...j->ij", I_h, I_l)

In [ ]:
I

In [ ]:
I.sum()

In [ ]:
C_l

In [ ]:
(C_h * C_l).sum(axis=0).softmax(axis=0)

In [ ]:
(C_h * C_l).sum(axis=1).softmax(axis=0)

In [ ]:
b = 1
n_z = 3
d_z = 4
n_x = 5
d_x = 2
#
d_i = 6
#
x = torch.rand(b, n_x, d_x)

model = CrossAttention(n_z, d_z, d_x, d_i)
out, C = model(x)
out.shape

In [ ]:
C

In [ ]:
S = torch.einsum("id, ...jd -> ...ij", q, k)
A_l = S.softmax(dim=1)
A_h = S.softmax(dim=2)
#
A = A_l * A_h
out = torch.einsum("...ij, ...jk -> ...ik", A, v)
out.shape

In [ ]:
class Attention(nn.Module):

    def __init__(self,
                 query_dim,
                 context_dim=None,
                 heads=8,
                 dim_head=64,
                 dropout=0.):
        super().__init__()
        inner_dim = dim_head * heads
        context_dim = helpers.default(context_dim, query_dim)

        self.scale = dim_head**-0.5
        self.heads = heads

        self.to_q = nn.Linear(query_dim, inner_dim, bias=False)
        self.to_kv = nn.Linear(context_dim, inner_dim * 2, bias=False)

        self.to_out = nn.Sequential(nn.Linear(inner_dim, query_dim),
                                    nn.Dropout(dropout))

    def forward(self, x, context=None, mask=None):
        h = self.heads

        q = self.to_q(x)
        context = helpers.default(context, x)
        k, v = self.to_kv(context).chunk(2, dim=-1)

        q, k, v = map(lambda t: rearrange(t, "b n (h d) -> (b h) n d", h=h),
                      (q, k, v))

        sim = einsum("b i d, b j d -> b i j", q, k) * self.scale

        if helpers.exists(mask):
            mask = rearrange(mask, "b ... -> b (...)")
            max_neg_value = -torch.finfo(sim.dtype).max
            mask = repeat(mask, "b j -> (b h) () j", h=h)
            sim.masked_fill_(~mask, max_neg_value)

        attn = sim.softmax(dim=1)

        out = einsum("b i j, b j d -> b i d", attn, v)
        out = rearrange(out, "(b h) n d -> b n (h d)", h=h)

        return self.to_out(out)